In [2]:
from math import *
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

import import_ipynb
from interp_1var import *

## Fonctions permettant l'interpolation de fonctions à 2 variables x et t

In [3]:
def Jacobian_X0_t(p, h, x, function, eps, t) : 
    f = function(x, h, t)
    f_max = np.max(f)
    f_eps = max(f_max, eps)
    I = np.zeros(2*p)

    I[:p] = b_prime_alpha(p)
    I[p:] = a_prime_beta(p)
  
    return np.sqrt(f_eps)*np.diag(I)

In [15]:
def a_t(x, alpha, h, p, function, eps, t):
    res = 0
    for i in range(p+1):
        som = (-1)**(i+p)* np.sqrt(max(function(alpha[i], h, t),eps) / alpha[i])
        mul = 1 
        for j in range(p+1) :
            if i != j :
                mul = mul * ((x - alpha[j]) / (alpha[i] - alpha[j])) 
        som  =  som* mul
        res += som
    return res

In [16]:
def b_t(x, beta, h, p, function, eps, t):
    res = 0
    for i in range(p+1) :
        som = (-1)**(i+p) * sqrt(max(function(beta[i], h, t), eps) / ( 1 - beta[i]))
        mul = 1 
        for j in range(p+1) : 
            if i != j :
                mul = mul * ((x - beta[j]) / (beta[i] - beta[j]))
        som = som* mul
        res += som
    return res

In [17]:
def teta_teta_t(x, h, p, function, eps, t) : 
    teta = np.zeros((2*p,1))
    alpha = x[:p]
    alpha_new = np.concatenate(( alpha, [[1]]))
    beta = x[p:]
    beta_new = np.concatenate(([[0]],beta))
    
    teta[p:] = a_t(beta, alpha_new, h, p, function, eps, t)
    teta[:p] = b_t(alpha, beta_new, h, p, function, eps, t)
    return teta 

In [18]:
def G_hat_t(x, h, p, function, eps, t) : 
    J = Jacobian_X0_t(p, h, x, function, eps, t)
    tet = teta_teta_t(x, h, p, function,eps, t)
    d = npl.solve(J,tet)
    return x - d

In [19]:
def Newton_Raphson_t(p, h, function, t, eps=1e-3)  :
    X0 = start_point(p)
    X = np.concatenate((X0[:p], X0[p+2:]))
    X_past = X + 2*eps
    err = 2*eps
    ERR = []
    i = 0
    while npl.norm(X-X_past)/npl.norm(X) > eps and i<20000:
        X_past = X
        X = G_hat_t(X, h, p, function, eps, t)
        i+=1
        ERR += [npl.norm(X - X_past)/npl.norm(X)]
    #print("Erreur Newton-Raphson : ", ERR[-1])
    return X, ERR, i

In [9]:
def interpol_t(p,h,function, t, eps=1e-12) :
    X, ERR, nit = Newton_Raphson_t(p, h, function, t, eps)
    X = np.concatenate((X[:p], [[1]], [[0]], X[p:]))
    
    xn = np.zeros((p+1,1))
    xn[:,0] = np.linspace(0,1,p+1)
    
    ar = X[:p+1]
    br = X[p+1:]
    
    A = a_t(xn, ar, h, p, function, eps, t)
    B = b_t(xn, br, h, p, function, eps, t)
    poln = polynom(xn, A, B)

    x = np.linspace(0,1,300)
    A_pol = a_t(x, ar, h, p ,function, eps, t)
    B_pol = b_t(x, br, h, p ,function, eps, t)
    pol = polynom(x, A_pol, B_pol)

    
    return poln, xn, ERR, pol, x, nit

In [10]:
def polynom_display_t(xn,poln, x, pol,h, real_func, t):
    x0 = np.linspace(0,1,100)
    
    plt.figure()
    plt.plot(x, pol, '-b')
    plt.plot(xn,poln, 'ob', label ="Points d'interpolation")
    plt.plot(x0, real_func(x0,h, t), '-r', label = "Fonction à interpoler")
    plt.legend()
    plt.show()   

In [11]:
def erreur_t(xn, poln, function, h, t) : 
    f = function(xn, h, t)
    dif = np.abs(f - poln)
    n = np.shape(dif)[0]
    dif = np.reshape(dif, (n))
    xn = np.reshape(xn, (n))
    som = 0
    for i in range(n-1) : 
        som += (xn[i+1]-xn[i]) * (dif[i+1]+dif[i]) / 2
    return som 

## Interpolation avec points initiaux différents

In [12]:
def Newton_Raphson_pts_init_t(p, h, function, X0, t, eps=1e-12)  :
    print(X0)
    X = np.concatenate((X0[:p], X0[p+2:]))
    X_past = X + 2*eps
    err = 2*eps
    ERR = []
    i = 0
    while npl.norm(X-X_past)/npl.norm(X) > eps and i<20000:
        X_past = X
        X = G_hat_t(X,h,p,function,eps,t)
        i+=1
        ERR += [npl.norm(X - X_past)/npl.norm(X)]
    print("Nombre d'itérations finales : ",i)
    #print("Erreur Newton-Raphson : ", ERR[-1])
    return X, ERR

In [13]:
def interpol_pts_init_t(p, h, function, X0, t, eps=1e-12) :
    X, ERR = Newton_Raphson_pts_init_t(p, h, function, X0, t, eps)
    X = np.concatenate((X[:p], [[1]], [[0]], X[p:]))
    
    xn = np.zeros((p+1,1))
    xn[:,0] = np.linspace(0,1,p+1)
    
    ar = X[:p+1]
    br = X[p+1:]
    
    A = a(xn, ar, h, p, function,eps )
    B = b(xn, br, h, p, function,eps )
    poln = polynom(xn, A, B)

    x = np.linspace(0,1,300)
    A_pol = a(x, ar, h, p ,function, eps)
    B_pol = b(x, br, h, p ,function, eps)
    pol = polynom(x, A_pol, B_pol)

    
    return poln, xn, ERR, pol, x